In [1]:
import pandas as pd
import numpy as np
import copy
from tqdm import tqdm
import sys
from rdkit import Chem
# sys.path.append('../Script-new/')
# from data_process import * 
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from common import *
import copy
import multiprocessing
from multiprocessing import Pool

### input and ouput

In [2]:
#input
mnxchem_file_path = '../../Data/database/mnx_chem_prop.tsv'
mnxreac_file_path = '../../Data/database/MNX_reac_prop.tsv'

#output
mnxmeta_smile_file_path = '../../Data/database/MNXmetabolite_smile.csv'
mnxreac_smile_file_path = '../../Data/database/MNXreaction_smile.csv'

In [3]:
conserveMets_MNX = {
    'currency': ['[H+]'],
    'pair': [
        'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O)([O-])OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)([O-])[O-])[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1', 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)([O-])OP(=O)([O-])OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)([O-])[O-])[C@@H]3O)[C@@H](O)[C@H]2O)c1',
        'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O)([O-])OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1', 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)([O-])OP(=O)([O-])OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](O)[C@@H]3O)[C@@H](O)[C@H]2O)c1',
        'Cc1cc2c(cc1C)N(C[C@H](O)[C@H](O)[C@H](O)COP(=O)([O-])[O-])c1[nH]c(=O)[nH]c(=O)c1N2', 'Cc1cc2nc3c(=O)[n-]c(=O)nc-3n(C[C@H](O)[C@H](O)[C@H](O)COP(=O)([O-])[O-])c2cc1C',
        'Cc1cc2c(cc1C)N(C[C@H](O)[C@H](O)[C@H](O)COP(=O)([O-])[O-])c1[n-]c(=O)[nH]c(=O)c1N2', 'Cc1cc2nc3c(=O)[nH]c(=O)nc-3n(C[C@H](O)[C@H](O)[C@H](O)COP(=O)(O)O)c2cc1C',
        'Cc1cc2nc3c(=O)[n-]c(=O)nc-3n(C[C@H](O)[C@H](O)[C@H](O)COP(=O)([O-])OP(=O)([O-])OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](O)[C@@H]3O)c2cc1C','Cc1cc2c(cc1C)N(C[C@H](O)[C@H](O)[C@H](O)COP(=O)([O-])OP(=O)([O-])OC[C@H]1O[C@@H](n3cnc4c(N)ncnc43)[C@H](O)[C@@H]1O)c1[nH]c(=O)[nH]c(=O)c1N2'
    ],
    'energy': [
        ['Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O','Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O','O=P([O-])([O-])O','[H]O[H]'],
        ['Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O','Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])[O-])[C@@H](O)[C@H]1O','O=P([O-])([O-])OP(=O)([O-])O','[H]O[H]'],
        ['Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O','Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])[O-])[C@@H](O)[C@H]1O','O=P([O-])([O-])O','[H]O[H]'],
        ['Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O','Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O'],
        ['Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O','Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])[O-])[C@@H](O)[C@H]1O'],
        ['Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O','Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])[O-])[C@@H](O)[C@H]1O'],
    ],
    'energy_mets':[
        ['ATP','ADP', 'phosphate','H2O'],#MNXM3/MNXM40333/MNXM9/WATER
        ['ATP','AMP','diphosphate','H2O'],
        ['ADP','AMP','phosphate','H2O'],
        ['ATP','ADP'],#MNXM3/MNXM40333/MNXM9/WATER
        ['ATP','AMP'],
        ['ADP','AMP'] 
    ]
}



def constructS(reaction):
    # Split the reaction into reactants and products
    sides = reaction.split('<=>')
    reactants = sides[0].split('+')
    products = sides[1].split('+')

    # Initialize empty lists for reactants and products
    S_reactants = [0] * len(reactants)
    S_products = [0] * len(products)
    mets = []

    # Parse reactants
    for i in range(len(reactants)):
        reactant = reactants[i].strip()
        coefficient, metabolite = parseCoeffMetabolite(reactant)
        S_reactants[i] = -coefficient  # Negative coefficient for reactants
        mets.append(metabolite)

    # Parse products
    for i in range(len(products)):
        product = products[i].strip()
        coefficient, metabolite = parseCoeffMetabolite(product)
        S_products[i] = coefficient  # Positive coefficient for products
        mets.append(metabolite)

    # Combine reactants and products into the stoichiometric matrix S
    S = S_reactants + S_products
    return S, mets

def parseCoeffMetabolite(string):
    # Split the string into coefficient and metabolite
    coefficient, metabolite = string.split(' ', 1)
    coefficient = float(coefficient)
    metabolite = metabolite.strip()
    return coefficient, metabolite

def is_all_zeros(lst):
    for element in lst:
        if element != 0:
            return False
    return True

def met_id_is_all_exist(met_list,mxnchem_id_list):
    for met in met_list:
        if met not in mxnchem_id_list:
            return False
    return True

def get_met_smiles_and_name(matrix_ids, mnxchem_df):
    smiles_list = []
    mnx_id_list = []
    for met in matrix_ids:
        met_data = mnxchem_df.loc[mnxchem_df['MNX_ID'] == met]
        mnx_id_list.append(met_data['name'].tolist()[0])
        smiles_list.append(met_data['SMILES'].tolist()[0])
    return mnx_id_list,smiles_list
def remove_comment_lines(df):
    df = df[~df['MNX_ID'].str.startswith('#')]
    df = df[~df['MNX_ID'].str.startswith('EMPTY')]
    return df
def supply_energy_metabolite(conserveMets_MNX_energy_pair, matrix_tmp):
    matrix_tmp_after_supply = matrix_tmp
    intersection = set(conserveMets_MNX_energy_pair) & set(matrix_tmp_after_supply['metsmiles'])
    if conserveMets_MNX_energy_pair[0] in matrix_tmp_after_supply['metsmiles'] and conserveMets_MNX_energy_pair[1] in matrix_tmp_after_supply['metsmiles'] and len(intersection) == 3:
        missing_metabolites = [metabolite for metabolite in conserveMets_MNX_energy_pair if metabolite not in matrix_tmp_after_supply['metsmiles']]
        # print(missing_metabolites)
        #miss water 
        if missing_metabolites[0] == '[H]O[H]': 
            #on the same side with atp
            if 'Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O' in matrix_tmp_after_supply['metsmiles']:
                ATP_index = matrix_tmp_after_supply['metsmiles'].index('Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O')#ATP
                coefficient = -matrix_tmp_after_supply['metcoef'][ATP_index]
            #on the same side with adp
            else:
                ADP_index = matrix_tmp_after_supply['metsmiles'].index('Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O')#ADP
                coefficient = -matrix_tmp_after_supply['metcoef'][ADP_index]

            matrix_tmp_after_supply['metcoef'].append(coefficient)
            matrix_tmp_after_supply['metMetaNetXID'].append('WATER')
            matrix_tmp_after_supply['mets'].append('H2O')
            matrix_tmp_after_supply['metsmiles'].append('[H]O[H]')    
        #miss O=P([O-])([O-])O  
        elif missing_metabolites[0] == 'O=P([O-])([O-])O':
            #opposite to atp
            if 'Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O' in matrix_tmp_after_supply['metsmiles']:
                ATP_index = matrix_tmp_after_supply['metsmiles'].index('Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O')#ATP
                coefficient = matrix_tmp_after_supply['metcoef'][ATP_index]
            #Opposite to adp
            else:
                ADP_index = matrix_tmp_after_supply['metsmiles'].index('Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O')#ADP
                coefficient = matrix_tmp_after_supply['metcoef'][ADP_index]

            matrix_tmp_after_supply['metcoef'].append(coefficient)
            matrix_tmp_after_supply['metMetaNetXID'].append('MNXM9')
            matrix_tmp_after_supply['mets'].append('phosphate')
            matrix_tmp_after_supply['metsmiles'].append('O=P([O-])([O-])O')
        #miss O=P([O-])([O-])OP(=O)([O-])O , opposite to atp
        elif missing_metabolites[0] == 'O=P([O-])([O-])OP(=O)([O-])O':
            ATP_index = matrix_tmp_after_supply['metsmiles'].index('Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O)([O-])OP(=O)([O-])[O-])[C@@H](O)[C@H]1O')#ATP
            coefficient = matrix_tmp_after_supply['metcoef'][ATP_index]
            matrix_tmp_after_supply['metcoef'].append(coefficient)
            matrix_tmp_after_supply['metMetaNetXID'].append('MNXM11')
            matrix_tmp_after_supply['mets'].append('diphosphate')
            matrix_tmp_after_supply['metsmiles'].append('O=P([O-])([O-])OP(=O)([O-])O')
        else:
            print('error')

    elif len(intersection)>4:
        print('error, energy intersection>4')
    
    else:pass

    return(matrix_tmp_after_supply)

def process_energy_metabolite(conserveMets_MNX_energy_pair, matrix_tmp , matrix_tmp_after_supply):
    intersection = set(conserveMets_MNX_energy_pair) & set(matrix_tmp['metsmiles'])
    if conserveMets_MNX_energy_pair[0] in matrix_tmp['metsmiles'] and conserveMets_MNX_energy_pair[1] in matrix_tmp['metsmiles'] and len(intersection) > 2:
        del_index = []
        #Find the deleted index
        for index, smiles in enumerate(matrix_tmp['metsmiles']):
            if smiles in intersection:
                del_index.append(index)
        #delete the supplemented metabolites        
        for key in matrix_tmp_after_supply.keys():
            matrix_tmp_after_supply[key] = [value for index, value in enumerate(matrix_tmp_after_supply[key]) if index not in del_index]
        return matrix_tmp_after_supply
    else:
        return matrix_tmp_after_supply



def process_currency_metabolite(conserveMets_MNX_currency, matrix_tmp):
    # Deal with the currency metabolite
    curr_met_tmp = list(set(conserveMets_MNX_currency) & set(matrix_tmp['metsmiles']))
    idx = [i for i, metsmile in enumerate(matrix_tmp['metsmiles']) if metsmile in curr_met_tmp]

    # Remove the currency metabolite from the matrix_tmp
    matrix_tmp['mets'] = [m for i, m in enumerate(matrix_tmp['mets']) if i not in idx]
    matrix_tmp['metsmiles'] = [sm for i, sm in enumerate(matrix_tmp['metsmiles']) if i not in idx]
    matrix_tmp['metMetaNetXID'] = [metid for i, metid in enumerate(matrix_tmp['metMetaNetXID']) if i not in idx]
    matrix_tmp['metcoef'] = [coef for i, coef in enumerate(matrix_tmp['metcoef']) if i not in idx]
    
    return matrix_tmp

def process_pair_metabolite(conserveMets_MNX_pair, matrix_tmp):
    # Deal with the pair metabolite
    curr_met_tmp = list(set(conserveMets_MNX_pair) & set(matrix_tmp['metsmiles']))
    if len(curr_met_tmp) >= 2:  # only remove when it is a pair
        idx = [i for i, metsmile in enumerate(matrix_tmp['metsmiles']) if metsmile in curr_met_tmp]

        # Remove the pair metabolite from the matrix_tmp
        matrix_tmp['mets'] = [m for i, m in enumerate(matrix_tmp['mets']) if i not in idx]
        matrix_tmp['metsmiles'] = [sm for i, sm in enumerate(matrix_tmp['metsmiles']) if i not in idx]
        matrix_tmp['metMetaNetXID'] = [metid for i, metid in enumerate(matrix_tmp['metMetaNetXID']) if i not in idx]
        matrix_tmp['metcoef'] = [coef for i, coef in enumerate(matrix_tmp['metcoef']) if i not in idx]

    return matrix_tmp

def get_matrix_tmp_and_matrix_tmp_dep(mnxreac_equation,conserveMets_MNX,mnxchem):#mnxreac_equation=row['equation']
    S, mets = constructS(mnxreac_equation)
    matrix_tmp = {}
    matrix_tmp['metcoef'] = S
    if not is_all_zeros(S):
        temp = [met.split('@', 1)[0] for met in mets]        
        matrix_tmp['metMetaNetXID'] = temp
        # print(matrix_tmp['metMetaNetXID'])
        # find met information
        if met_id_is_all_exist(matrix_tmp['metMetaNetXID'],mnxchem['MNX_ID'].to_list()):
            matrix_tmp['mets'],matrix_tmp['metsmiles'] = get_met_smiles_and_name(matrix_tmp['metMetaNetXID'],mnxchem)
            # print('org met',matrix_tmp['mets'])
            # print(matrix_tmp['metsmiles'])
            matrix = copy.deepcopy(matrix_tmp)
            # print('no supply matrix1',matrix)
            # energy
            #Complement the energy reaction first
            for conserveMets_MNX_energy_pair in conserveMets_MNX['energy']:
                matrix_tmp_after_supply = supply_energy_metabolite(conserveMets_MNX_energy_pair, matrix_tmp) 
            # print('supply met',matrix_tmp_after_supply['mets'])
            # print('supply matrix',matrix_tmp_after_supply)

            # print('no supply matrix2',matrix)
            #delete energy pair
            for conserveMets_MNX_energy_pair in conserveMets_MNX['energy']:                      
                matrix_tmp_dep = process_energy_metabolite(conserveMets_MNX_energy_pair, matrix, matrix_tmp_after_supply)

            # print('dep met',matrix_tmp_dep['mets'])       

            # deal with the currency metabolite and redox stuff
            matrix_tmp_dep = process_currency_metabolite(conserveMets_MNX['currency'],matrix_tmp_dep)
            # print(matrix_tmp_dep)

            # pair
            matrix_tmp_dep = process_pair_metabolite(conserveMets_MNX['pair'], matrix_tmp_dep)
            # print('dep met',matrix_tmp_dep['mets'])
        # else:print('met_id_is_not_all_exist',mnxreac_equation)
        return matrix, matrix_tmp_dep
    else: 
        print('S=0',mnxreac_equation)

def get_substrate_smiles(matrix_tmp_dep):
    substrates = []
    for i, coef in enumerate(matrix_tmp_dep['metcoef']):
        if coef < 0:
            substrates.extend([matrix_tmp_dep['metsmiles'][i]] * int(abs(coef)))
    substrate_smiles = '.'.join(substrates)
    return substrate_smiles

def get_product_smiles(matrix_tmp_dep):
    products = []
    for i, coef in enumerate(matrix_tmp_dep['metcoef']):
        if coef > 0:
            products.extend([matrix_tmp_dep['metsmiles'][i]] * int(abs(coef)))
    product_smiles = '.'.join(products)
    return product_smiles

def get_equ_name(matrix_tmp_dep):
    left_components = []
    right_components = []

    for i, coef in enumerate(matrix_tmp_dep['metcoef']):
        if coef != 0:
            if coef == 1:
                right_components.append(matrix_tmp_dep['mets'][i])
            elif coef == -1:
                left_components.append(matrix_tmp_dep['mets'][i])
            elif coef < 0:
                left_component = str(abs(coef)) + ' ' + matrix_tmp_dep['mets'][i]
                left_components.append(left_component)
            else:
                right_component = str(coef) + ' ' + matrix_tmp_dep['mets'][i]
                right_components.append(right_component)

    left_side = ' + '.join(left_components)
    right_side = ' + '.join(right_components)

    # # Handle the case when one side is empty
    # if not left_side:
    #     equ_name = right_side
    # elif not right_side:
    #     equ_name = left_side
    # else:
    equ_name = left_side + ' <=> ' + right_side

    return equ_name


def get_equ_smiles(matrix_tmp_dep):
    substrate_smiles = get_substrate_smiles(matrix_tmp_dep)
    product_smiles = get_product_smiles(matrix_tmp_dep)
    equ_smiles = substrate_smiles + '>>' + product_smiles
    return equ_smiles


def get_deprecated_equ_and_name(matrix,matrix_tmp_dep):
    del_mets = [met for met in matrix['mets'] if met not in matrix_tmp_dep['mets']]
    del_mets_indexes = [matrix['mets'].index(met) for met in del_mets]
    matrix_del = {
        'metcoef': [matrix['metcoef'][index] for index in del_mets_indexes],###----
        'metMetaNetXID': [matrix['metMetaNetXID'][index] for index in del_mets_indexes],
        'mets': [matrix['mets'][index] for index in del_mets_indexes],
        'metsmiles': [matrix['metsmiles'][index] for index in del_mets_indexes]
    }

    add_mets = [met for met in matrix_tmp_dep['mets'] if met not in matrix['mets']]
    add_mets_indexes = [matrix_tmp_dep['mets'].index(met) for met in add_mets]
    matrix_add = {
        'metcoef': [-matrix_tmp_dep['metcoef'][index] for index in add_mets_indexes],#####+++++
        'metMetaNetXID': [matrix_tmp_dep['metMetaNetXID'][index] for index in add_mets_indexes],
        'mets': [matrix_tmp_dep['mets'][index] for index in add_mets_indexes],
        'metsmiles': [matrix_tmp_dep['metsmiles'][index] for index in add_mets_indexes]
    } 

    matrix_deprecated = {}
    for key in matrix_del.keys():
        matrix_deprecated[key] = matrix_del.get(key, []) + matrix_add.get(key, [])

    return get_equ_name(matrix_deprecated),get_equ_smiles(matrix_deprecated)

def process_MetNetx_reaction(mnxreac_file_path,mnxreac_smile_file_path,mnxchem_file_path):
    mnxchem = pd.read_csv(mnxchem_file_path, delimiter='\t', header=None) 
    mnxchem = mnxchem.loc[1:,:]
    mnxchem.columns = ['MNX_ID', 'name', 'REFERENCE', 'formula', 'charge', 'mass', 'standardInChI', 'standardInChIKey', 'SMILES']
    mnxchem = remove_comment_lines(mnxchem)
    mnxchem = mnxchem.dropna(subset=['SMILES'])

    mnxreac = pd.read_csv(mnxreac_file_path, delimiter='\t', header=None)
    mnxreac.columns = ['MNX_ID', 'equation', 'REFERENCE', 'classifs', 'is_balanced', 'is_transport']#ID	mnx_equation	reference	classifs	is_balanced	is_transport
    mnxreac = mnxreac[~mnxreac['MNX_ID'].str.startswith('#')]
    mnxreac = mnxreac[~mnxreac['MNX_ID'].str.startswith('EMPTY')]
    # mnxreac = remove_comment_lines(mnxreac)
    mnxreac = mnxreac[(mnxreac['is_balanced']=='B')&(mnxreac['is_transport']!='T')]
    mnxreac['equation'] = mnxreac['equation'].apply(lambda x:x.replace(' = ', ' <=> '))
    for index,row in tqdm(mnxreac.iterrows(),total=len(mnxreac)):
        # print(row['equation'],conserveMets_MNX,mnxchem)
        try:
            matrix,matrix_tmp_dep = get_matrix_tmp_and_matrix_tmp_dep(row['equation'],conserveMets_MNX,mnxchem)
            mnxreac.loc[index,'substrate_smiles']=get_substrate_smiles(matrix_tmp_dep)
            mnxreac.loc[index,'product_smiles']=get_product_smiles(matrix_tmp_dep)    
            mnxreac.loc[index,'equ_name']=get_equ_name(matrix_tmp_dep)
            mnxreac.loc[index,'equ_smiles']=get_equ_smiles(matrix_tmp_dep)
            mnxreac.loc[index,'deprecated_equ'],mnxreac.loc[index,'deprecated_equ_smiles'] = get_deprecated_equ_and_name(matrix,matrix_tmp_dep)
        except:
            pass
        # deprecated_equ, deprecated_equ_smiles = get_deprecated_equ_and_name(matrix,matrix_tmp_dep) 
    mnxreac = mnxreac[mnxreac['equ_smiles']!='>>']
    mnxreac.to_csv(mnxreac_smile_file_path,index=None) 

def process_MetNetx_metabolite(mnxchem_file_path,mnxmeta_smile_file_path):
    mnxchem = pd.read_csv(mnxchem_file_path, delimiter='\t', header=None) 
    mnxchem = mnxchem.loc[1:,:]
    mnxchem.columns = ['MNX_ID', 'name', 'REFERENCE', 'formula', 'charge', 'mass', 'standardInChI', 'standardInChIKey', 'SMILES']
    mnxchem = mnxchem[~mnxchem['MNX_ID'].str.startswith('#')]
    mnxchem = mnxchem[~mnxchem['MNX_ID'].str.startswith('EMPTY')]
    mnxchem = mnxchem.dropna(subset=['SMILES'])
    mnxchem = mnxchem[['MNX_ID', 'name', 'formula', 'charge', 'mass', 'SMILES']]
    mnxchem = mnxchem.drop_duplicates(subset='SMILES')
    mnxchem = mnxchem[~mnxchem['SMILES'].str.contains('\*')] #escape character
    print(mnxchem.shape)
    mnxchem.to_csv(mnxmeta_smile_file_path,index=None)

### 1. get reactions from MNX

In [4]:
process_MetNetx_reaction(mnxreac_file_path,mnxreac_smile_file_path,mnxchem_file_path)
# process_MetNetx_reaction(mnxreac_file_path,mnxreac_smile_file_path,mnxchem_file_prepare_path)

/home/wuke/anaconda3/envs/kcat/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
100%|██████████| 23583/23583 [2:13:18<00:00,  2.95it/s]  


### 2. get metabolites from MNX

In [5]:
process_MetNetx_metabolite(mnxchem_file_path,mnxmeta_smile_file_path)
# process_MetNetx_metabolite(mnxchem_file_prepare_path,mnxmeta_smile_file_path)

/home/wuke/anaconda3/envs/kcat/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


(1049196, 6)
